# Unit 4 Securing Your MCP Server with API Key Authentication

Welcome back\! In the previous lesson, you learned how to mount your **MCP server** inside a **FastAPI** application. This allowed you to serve both your agent tools and regular web API endpoints from a single, scalable app. By combining FastAPI’s flexibility with the power of the MCP server, you are now able to build modern applications that can handle both standard API requests and agent tool calls in one place.

As you continue to build more advanced and production-ready systems, it becomes important to think about security. In real-world scenarios, you do not want just anyone to access your agent tools or sensitive endpoints. Without proper protection, someone could misuse your tools, overload your server, or even access private data. That is why, in this lesson, you will learn how to secure your MCP server using **API key authentication**. This will help you control who can access your tools and ensure that only trusted agents or users are allowed in.

## API Key Authentication Overview

**API key authentication** is a simple but effective way to protect your web services. An API key is a secret value — usually a long string — that you give to trusted clients. When a client (like your agent) wants to use your service, it must include this key in its request, usually in a special HTTP header or as a query parameter.

When your server receives a request, it checks for the API key and verifies that it matches the expected value. If the key is missing or incorrect, the server rejects the request. This approach is widely used for internal tools, microservices, and agent integrations because it is easy to implement and works well for server-to-server communication.

By adding API key authentication to your MCP server, you ensure that only clients who know the secret key can access your tools. This is a strong first step toward securing your application.

## Middleware and Authentication

If you haven’t worked with middleware before, here’s a quick explanation. In web frameworks like FastAPI, **middleware** is a special layer that sits between the client and your application’s endpoints. Every incoming request passes through the middleware before it reaches your route handlers, and every response passes back through the middleware before it goes to the client.

Middleware is commonly used for tasks that need to happen on every request, such as logging, handling CORS, or — as in this lesson — checking authentication credentials like API keys. By placing your authentication logic in middleware, you ensure that all requests are checked for a valid API key before any sensitive code or data is accessed. This keeps your security logic centralized, consistent, and easy to maintain.

To add API key authentication to your FastAPI app, we will use a middleware to inspect each request for the presence and validity of an API key, and possibly reject requests before they reach your endpoints.

## Implementing the Authentication Middleware

In FastAPI, you can add middleware by creating a class that defines what should happen for each request, and then registering that class with your app. This makes it straightforward to enforce security policies across your entire application.

Here is an example of an authentication middleware you can use:

```python
from starlette.middleware.base import BaseHTTPMiddleware
from fastapi.responses import JSONResponse
from starlette.requests import Request

# The expected API key value for authentication
API_KEY = "super_secret_value"
# The HTTP header name where the API key should be supplied
HEADER = "X-API-Key"

class ApiKeyMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        # Try to get the API key from the header
        supplied = request.headers.get(HEADER)

        # If the supplied API key does not match, return a 401 Unauthorized response
        if supplied != API_KEY:
            return JSONResponse(content={"detail": "Invalid API key"}, status_code=401)

        # If the API key is valid, continue processing the request
        return await call_next(request)
```

This middleware checks every incoming request for the correct API key. It looks for the key in the `X-API-Key` header. If the key is missing or incorrect, it returns a `401 Unauthorized` response with a simple error message. If the key is valid, the request continues as normal, allowing the client to access your regular API endpoints as well as use the MCP tools provided by your server.

For example, if a client sends a request without the correct key, the response will look like this:

```json
{
  "detail": "Invalid API key"
}
```

This makes it clear to the client that authentication is required.

## Protecting Only the Mounted MCP Server

If you only want to secure your MCP tools and leave your regular API endpoints open, you can easily do this by checking the route where you mounted the MCP server. For example, if you mounted your MCP server at `/mcp`, you can have your authentication middleware require an API key only for requests to `/mcp` and its subpaths.

Here’s how you can update your middleware to protect just the MCP route:

```python
from starlette.middleware.base import BaseHTTPMiddleware
from fastapi.responses import JSONResponse
from starlette.requests import Request

# The expected API key value for authentication
API_KEY = "super_secret_value"
# The HTTP header name where the API key should be supplied
HEADER = "X-API-Key"
# The route prefix for your MCP server (change if you mount elsewhere)
MCP_PREFIX = "/mcp"

class ApiKeyMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        # Check if the request path starts with the MCP prefix
        if request.url.path.startswith(MCP_PREFIX):
            # Try to get the API key from the header
            supplied = request.headers.get(HEADER)
            # If the supplied API key does not match, return a 401 Unauthorized response
            if supplied != API_KEY:
                return JSONResponse(
                    {"detail": "Invalid API key"},
                    status_code=401
                )
        # For all other routes, or if the API key is valid, continue processing the request
        return await call_next(request)
```

With this setup, only requests to `/mcp` (and anything under it, like `/mcp/sse` or `/mcp/messages`) will need to include the correct API key. All your other routes, such as `/` and `/items`, will remain open and won’t require authentication.

This approach is handy when you want to keep your agent tools protected, but still allow easy access to your regular API endpoints. If you ever change the route where you mount your MCP server, just update the `MCP_PREFIX` variable in your middleware.

## Integrating the Middleware into the FastAPI App

Now that we have our authentication middleware, we need to add it to your FastAPI application. This is done by registering the middleware when creating the FastAPI app. Here is how you can do it:

```python
import uvicorn
from fastapi import FastAPI
from starlette.middleware import Middleware
from auth import ApiKeyMiddleware  # Import the created middleware
from mcp_server import mcp
from shopping_list import ShoppingListService

# Create the FastAPI app and add middleware
app = FastAPI(
    middleware=[Middleware(ApiKeyMiddleware)]
)
# Create a shopping list service instance
shopping_list = ShoppingListService()

@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/mcp'
app.mount('/mcp', mcp.sse_app())

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

In this setup, how your `ApiKeyMiddleware` works depends on how you configure it:

  * **If you protect the whole application:** Every request—whether it’s for a regular endpoint like `/` or `/items`, or for the MCP server at `/mcp`—must include the correct API key. Any request without the right key will be blocked before it reaches your endpoints or tools.
  * **If you protect only the MCP server route:** Only requests to `/mcp` and its subpaths will require the API key. Regular endpoints like `/` and `/items` will remain open and accessible without authentication. Requests to the MCP server without the correct key will be blocked, but everything else will work as usual.

This gives you flexibility: you can secure your entire application or just your MCP tools, depending on your needs, all within the same FastAPI app.

## Configuring the Agent for Secure Access (agent.py)

With your server now protected by API key authentication, your agent must supply the correct key when connecting. This is done by adding the API key to the request headers when the agent connects to the MCP server.

Here is how we can do this in the agent script:

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse

async def main():
    # Setup the SSE server parameters with the API key
    server_params = {
        "url": "http://localhost:3000/mcp/sse",
        "headers": {"X-API-Key": "super_secret_value"},
    }
    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Create and run the agent...
        pass # Placeholder for agent creation and running

if __name__ == "__main__":
    asyncio.run(main())
```

In this example, the agent includes the `X-API-Key` header with the value `super_secret_value` when connecting to the MCP server. This matches the key expected by your middleware. As a result, the client is able to authenticate and the agent can use the tools provided by your MCP server.

If the client does not supply the correct key, it will receive a `401 Unauthorized` error and the agent will not be able to access the tools.

## Summary & Preparation for Practice

In this lesson, you learned how to secure your MCP server by adding API key authentication using middleware in FastAPI. You saw how the middleware checks each request for the correct API key and blocks unauthorized access. You also learned how to configure your agent to supply the API key when connecting, ensuring a secure communication channel between your agent and the MCP server.

By following these steps, you have taken an important step toward building secure, production-ready applications. You are now ready to practice these skills in hands-on exercises, where you will implement and test API key authentication yourself. Great job integrating security into your advanced MCP server and agent setup\!

## Securing MCP Tools with Middleware

It's time to secure your MCP server using API key authentication! 🔒

You’ve already been provided with an auth.py file that contains a ready-to-use ApiKeyMiddleware class. This middleware will protect your server by checking every incoming request for a valid API key, ensuring that only authorized clients can access your endpoints and MCP tools.

Your task is to integrate this middleware into your FastAPI app. Here’s what you need to do:

Import ApiKeyMiddleware from the auth.py file.
Add the middleware to your FastAPI app by updating the app creation line, using the starlette.middleware.Middleware class.
Once you’ve added the middleware, click the Run button. You should see that the agent and its MCP client are now unable to connect and receive a 401 Unauthorized error, confirming that your server is protected and only accessible to clients with the correct API key.


```python
# main.py
import uvicorn
from fastapi import FastAPI
from starlette.middleware import Middleware
from shopping_list import ShoppingListService
from mcp_server import mcp
# TODO: Import ApiKeyMiddleware from auth.py


# TODO: Create the FastAPI app and add ApiKeyMiddleware as middleware
app = FastAPI()

# Create a shopping list service instance
shopping_list = ShoppingListService()


@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/mcp'
app.mount('/mcp', mcp.sse_app())

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

# auth.py
from starlette.middleware.base import BaseHTTPMiddleware
from fastapi.responses import JSONResponse
from starlette.requests import Request

# The expected API key value for authentication
API_KEY = "super_secret_value"

# The HTTP header name where the API key should be supplied
HEADER = "X-API-Key"


class ApiKeyMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        # Try to get the API key from the header
        supplied = request.headers.get(HEADER)
        
        # If the supplied API key does not match, return a 401 Unauthorized response
        if supplied != API_KEY:
            return JSONResponse(content={"detail": "Invalid API key"}, status_code=401)
            
        # If the API key is valid, continue processing the request
        return await call_next(request)

# agent.py
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse


async def main():
    # Setup the SSE server parameters WITHOUT the API key
    server_params = {"url": "http://localhost:3000/mcp/sse"}

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

To secure your MCP tools with middleware, you need to modify the `main.py` file.

Here's the corrected `main.py` code:

```python
# main.py
import uvicorn
from fastapi import FastAPI
from starlette.middleware import Middleware
from shopping_list import ShoppingListService
from mcp_server import mcp
from auth import ApiKeyMiddleware  # Import ApiKeyMiddleware from auth.py


# Create the FastAPI app and add ApiKeyMiddleware as middleware
app = FastAPI(
    middleware=[Middleware(ApiKeyMiddleware)]
)

# Create a shopping list service instance
shopping_list = ShoppingListService()


@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/mcp'
app.mount('/mcp', mcp.sse_app())

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

```

## Authenticating Your Agent Connection

With your MCP server now secured using API key authentication, it’s time to update the agent script so it can connect to the protected server. 🔑

Your task is to update the agent.py file so that the server connection settings include the correct API key in the request headers. This will enable the agent to authenticate and communicate with the server.

In the server_params dictionary, add a "headers" key.
Set the value of "headers" to a dictionary containing the "X-API-Key" header with the correct API key value ("super_secret_value").
Once you’ve made this change, run the agent again. If everything is set up properly, your agent should be able to connect and use the protected MCP tools.

```python
# agent.py
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse


async def main():
    # Setup the SSE server parameters WITHOUT the API key
    server_params = {
        "url": "http://localhost:3000/mcp/sse"
        # TODO: Add the "headers" key here with the correct API key in the "X-API-Key" header
    }

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

# auth.py
from starlette.middleware.base import BaseHTTPMiddleware
from fastapi.responses import JSONResponse
from starlette.requests import Request

# The expected API key value for authentication
API_KEY = "super_secret_value"

# The HTTP header name where the API key should be supplied
HEADER = "X-API-Key"


class ApiKeyMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        # Try to get the API key from the header
        supplied = request.headers.get(HEADER)
        
        # If the supplied API key does not match, return a 401 Unauthorized response
        if supplied != API_KEY:
            return JSONResponse(content={"detail": "Invalid API key"}, status_code=401)
            
        # If the API key is valid, continue processing the request
        return await call_next(request)

# main.py
import uvicorn
from fastapi import FastAPI
from starlette.middleware import Middleware
from auth import ApiKeyMiddleware
from shopping_list import ShoppingListService
from mcp_server import mcp

# Create the FastAPI app and add middleware
app = FastAPI(
    middleware=[Middleware(ApiKeyMiddleware)]
)

# Create a shopping list service instance
shopping_list = ShoppingListService()


@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/mcp'
app.mount('/mcp', mcp.sse_app())

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

```

To authenticate your agent connection, you need to add the `headers` key with the API key to the `server_params` dictionary in `agent.py`.

Here's the corrected `agent.py` code:

```python
# agent.py
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse


async def main():
    # Setup the SSE server parameters WITH the API key
    server_params = {
        "url": "http://localhost:3000/mcp/sse",
        "headers": {"X-API-Key": "super_secret_value"} # Added the "headers" key with the API key
    }

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

## Selective Protection for MCP Tools

Now, let’s make your security setup a bit more flexible.

In many real-world apps, you might want to protect only certain parts of your API — like your MCP tools — while keeping other endpoints open to everyone. Your task is to update the authentication middleware so that only requests to the MCP server (the /mcp route and anything under it) require an API key. All other routes, such as /items/, should remain public and not require authentication.

To complete this exercise:

Add a constant for the MCP route prefix (for example, MCP_PREFIX = "/mcp").
Update the middleware so it checks for the API key only if the request path starts with the MCP prefix.
Make sure requests to other routes are not blocked by the middleware.
This is a common pattern for protecting sensitive tools while keeping your main API user-friendly. Give it a try and see how you can control access to just the parts of your app that need it!

```python
# agent.py
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse


async def main():
    # Setup the SSE server parameters with the API key
    server_params = {
        "url": "http://localhost:3000/mcp/sse",
        "headers": {"X-API-Key": "super_secret_value"},
    }

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())
    
# auth.py
from starlette.middleware.base import BaseHTTPMiddleware
from fastapi.responses import JSONResponse
from starlette.requests import Request

# The expected API key value for authentication
API_KEY = "super_secret_value"

# The HTTP header name where the API key should be supplied
HEADER = "X-API-Key"

# TODO: Add a constant for the prefix where the MCP server is mounted

class ApiKeyMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        # TODO: Check if the request path starts with the mounted prefix
        # If it does, only then check for the API key in the header

        # Try to get the API key from the header
        supplied = request.headers.get(HEADER)
        
        # If the supplied API key does not match, return a 401 Unauthorized response
        if supplied != API_KEY:
            return JSONResponse(content={"detail": "Invalid API key"}, status_code=401)
            
        # If the API key is valid, continue processing the request
        return await call_next(request)

# main.py
import uvicorn
from fastapi import FastAPI
from starlette.middleware import Middleware
from auth import ApiKeyMiddleware
from shopping_list import ShoppingListService
from mcp_server import mcp  # Import the MCP server with tools

# Create the FastAPI app and add middleware
app = FastAPI(
    middleware=[Middleware(ApiKeyMiddleware)]
)

# Create a shopping list service instance
shopping_list = ShoppingListService()


@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/mcp'
app.mount('/mcp', mcp.sse_app())

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

```